In [14]:
#figure out what files were skipped for/are missing from results

import os
import time
from os import listdir
from os.path import isfile, join

direct2= 'C:/Users/pouls/Documents/NSP3_workingstuff/Human/splits/'
direct3= 'C:/Users/pouls/Documents/NSP3_workingstuff/Human_results/'
files = [f for f in listdir(direct2) if isfile(join(direct2,f)) ]
results =  [f for f in listdir(direct3) if isfile(join(direct3,f)) ]
files.sort()
pot_res=[]
for i in range(215):
    temp='results'+str(i)+'.csv'
    pot_res.append(temp)
missing = [name for name in pot_res if name not in results]
print(missing)
filestorun=[]
for i in missing:
    temp= i[:-4]
    temp=int(temp[7:])
    filestorun.append(files[temp])

# print(files)
# print(results)

['results50.csv', 'results52.csv', 'results53.csv', 'results59.csv', 'results179.csv', 'results188.csv', 'results194.csv']


In [15]:
filestorun

['human1039.fasta',
 'human1040.fasta',
 'human1041.fasta',
 'human1047.fasta',
 'human1155.fasta',
 'human1163.fasta',
 'human1169.fasta']

In [4]:
pot_res=[]
for i in range(225):
    temp='results'+str(i)+'.csv'
    pot_res.append(temp)

In [11]:
temp=missing[1][:-4]
temp=temp[7:]
temp

'1'